# Hyperwave API Workflow

This notebook demonstrates the **API workflow** for FDTD photonics simulations.
The API workflow uses high-level functions that handle structure creation internally.

**For the local workflow** (step-by-step structure creation with full control),
see `local_workflow.ipynb`.

**Steps:**
1. Build recipe from GDSFactory component (API)
2. Build monitors
3. Compute frequency band
4. Solve waveguide mode
5. Run GPU simulation (uses credits)
6. Analyze results

## Installation

In [ ]:
# Install hyperwave-community with gdsfactory
# Note: You may see dependency warnings - these are harmless and expected in Colab
%pip install "hyperwave-community[gds] @ git+https://github.com/spinsphotonics/hyperwave-community.git@main" -q

# Restart runtime if you see numpy import errors
# (Colab menu: Runtime -> Restart runtime)
import sys
if 'google.colab' in sys.modules:
    print("After installation, go to Runtime -> Restart runtime, then continue from the next cell")

## Configure API

In [ ]:
import hyperwave_community as hwc

# Configure and validate API key
account = hwc.configure_api(
    api_key="your-api-key-here",
    # api_url defaults to production Modal endpoint
)

hwc.get_account_info();

## Step 1: Build Recipe (API)

In [ ]:
# Component settings
COMPONENT_NAME = "mmi2x2_with_sbend"
EXTENSION_LENGTH = 2.0
COMPONENT_KWARGS = None

# Resolution settings
RESOLUTION_NM = 20  # Grid cell size in nanometers

# Alternative: Calculate from cells per wavelength
# WAVELENGTH_UM = 1.55
# N_CORE = 3.48
# CELLS_PER_WL = 25
# RESOLUTION_NM = (WAVELENGTH_UM * 1000) / CELLS_PER_WL / N_CORE  # = 17.8nm

print(f"Resolution: {RESOLUTION_NM}nm")

# Material properties
N_CORE = 3.48
N_CLAD = 1.4457
WG_HEIGHT_UM = 0.22
TOTAL_HEIGHT_UM = 4.0
PADDING = (100, 100, 0, 0)
DENSITY_RADIUS = 3
VERTICAL_RADIUS = 2.0

# Build recipe via API (single call)
recipe_result = hwc.build_recipe(
    component_name=COMPONENT_NAME,
    component_kwargs=COMPONENT_KWARGS,
    extension_length=EXTENSION_LENGTH,
    resolution_nm=RESOLUTION_NM,
    n_core=N_CORE,
    n_clad=N_CLAD,
    wg_height_um=WG_HEIGHT_UM,
    total_height_um=TOTAL_HEIGHT_UM,
    padding=PADDING,
    density_radius=DENSITY_RADIUS,
    vertical_radius=VERTICAL_RADIUS,
)

print(f"Structure dimensions: {recipe_result['dimensions']}")
print(f"Ports: {list(recipe_result['port_info'].keys())}")

## Step 2: Build Monitors

In [ ]:
SOURCE_PORT = "o1"

monitor_result = hwc.build_monitors(
    port_info=recipe_result['port_info'],
    dimensions=recipe_result['dimensions'],
    source_port=SOURCE_PORT,
    resolution_um=recipe_result['resolution_um'],
    structure_recipe=recipe_result['recipe'],
    show_structure=True,
)

print(f"Monitors: {list(monitor_result['monitor_names'].keys())}")
print(f"Source port: {monitor_result['source_port_name']}")
print(f"Source position: x={monitor_result['source_position']}")

## Step 3: Compute Frequency Band

In [ ]:
WL_CENTER_UM = 1.55
N_FREQS = 1

freq_result = hwc.compute_freq_band(
    wl_min_um=WL_CENTER_UM,
    wl_max_um=WL_CENTER_UM,
    n_freqs=N_FREQS,
    resolution_um=recipe_result['resolution_um'],
)

print(f"Frequency band: {freq_result['freq_band']}")
print(f"Wavelengths: {freq_result['wavelengths_um']}")

## Step 4: Solve Waveguide Mode

In [ ]:
MODE_NUM = 0

source_result = hwc.solve_mode_source(
    density_core=recipe_result['density_core'],
    density_clad=recipe_result['density_clad'],
    source_x_position=monitor_result['source_position'],
    mode_bounds=monitor_result['mode_bounds'],
    layer_config=recipe_result['layer_config'],
    eps_values=recipe_result['eps_values'],
    freq_band=freq_result['freq_band'],
    mode_num=MODE_NUM,
    show_mode=True,
)

print(f"Source field shape: {source_result['source_field'].shape}")
print(f"Source offset: {source_result['source_offset']}")

In [ ]:
# Estimate cost before running (free, no auth required)
dims = recipe_result['dimensions']
cost = hwc.estimate_cost(
    grid_points=dims[0] * dims[1] * dims[2],
    max_steps=20000,
    gpu_type="B200",
)
print(f"Estimated time: {cost['estimated_seconds']:.0f}s")
print(f"Estimated cost: ${cost['estimated_cost_usd']:.2f}")

## Step 5: Run Simulation

In [ ]:
NUM_STEPS = 20000
GPU_TYPE = "B200"

absorber_params = hwc.get_optimized_absorber_params(
    resolution_nm=RESOLUTION_NM,
    wavelength_um=WL_CENTER_UM,
    structure_dimensions=recipe_result['dimensions'],
)
print(f"Absorber params: {absorber_params}")

results = hwc.run_simulation(
    device_type=COMPONENT_NAME,
    recipe_result=recipe_result,
    monitor_result=monitor_result,
    freq_result=freq_result,
    source_result=source_result,
    num_steps=NUM_STEPS,
    gpu_type=GPU_TYPE,
    absorption_widths=absorber_params['absorption_widths'],
    absorption_coeff=absorber_params['absorber_coeff'],
    convergence="default",
)

print(f"\nSimulation time: {results['sim_time']:.1f}s")
print(f"Total execution time: {results['total_time']:.1f}s")

## Step 6: Analyze Results

In [ ]:
transmission = hwc.analyze_transmission(
    results,
    input_monitor="Input_o1",
    output_monitors=["Output_o3", "Output_o4"],
)

print(f"Input power: {transmission['power_in']:.4f}")
print(f"Total transmission: {transmission['total_transmission']:.4f}")
print(f"Excess loss: {transmission['excess_loss_dB']:.2f} dB")

In [ ]:
# === FIELD INTENSITY VISUALIZATION ===
# Plot |E|^2 from xy_mid monitor
import numpy as np
import matplotlib.pyplot as plt

monitor_data = results['monitor_data']
xy_mid_data = monitor_data['xy_mid']  # Shape: (N_freq, 6, Lx, Ly, 1)
print(f"xy_mid shape: {xy_mid_data.shape}")

# Calculate |E|^2 = |Ex|^2 + |Ey|^2 + |Ez|^2
E_fields = xy_mid_data[0, 0:3, :, :, 0]  # First freq, E components, squeeze Z
E_intensity = np.sum(np.abs(E_fields)**2, axis=0)

# Plot field intensity (pixel coordinates)
plt.figure(figsize=(14, 5))
plt.imshow(
    E_intensity.T,
    origin='lower',
    cmap='jet',
    aspect='equal'
)
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.title("|E|^2 Field Intensity")
plt.colorbar(label='|E|^2')
plt.tight_layout()
plt.show()
